# Generate ChordPro PDFs

- input/songs/sections/
    - Testlieder
- output/chordpro-pdfs/sections/
    - Testlieder

In [ ]:
# https://ipython.readthedocs.io/en/stable/config/options/terminal.html#configtrait-InteractiveShell.ast_node_interactivity
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # last_expr, all

In [ ]:
import os
import sys
import fnmatch
import subprocess
import pandas as pd

import pprint

In [ ]:
%load_ext autoreload
%autoreload 1

#from chordprosongbook import songscanner
# use this type of import for autoreload:
%aimport chordprosongbook.songscanner
%aimport chordprosongbook.pdfmaker



In [ ]:
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
path_input_song_sections = os.path.normpath("../data/input/song-sections/")
path_input_song_sections
scanner = chordprosongbook.songscanner.SongScanner(path_input_song_sections)

In [ ]:
scanner.detect_song_sections()
scanner.song_sections

In [ ]:
scanner.detect_songs_of_sections(subfolder="chordpro-songs",extension="cho")
scanner.detect_songs_of_sections(subfolder="pdf-songs",extension="pdf")
scanner.song_sections

In [ ]:
pp.pprint([section['name'] for section in scanner.song_sections])

In [ ]:
# https://stackoverflow.com/questions/40972805/python-capture-contents-inside-curly-braces/40972959
# https://stackoverflow.com/questions/11310567/python-re-match-string-in-a-file/11310926

scanner.extract_chordpro_metadata()
           
# pp.pprint(scanner.song_sections[0]["songs"])

In [ ]:
scanner.song_sections

### info/tips

- subprocess tutorial: https://youtu.be/2Fp1N6dof0Y
- using chordpro cli: https://www.chordpro.org/chordpro/using-chordpro/

In [ ]:
# song_sections

In [ ]:
path_song_sections_pdf_includes = os.path.normpath("../data/generated-pdf-includes-do-not-edit/")
pdfmaker = chordprosongbook.pdfmaker.PdfMaker(scanner.song_sections,path_song_sections_pdf_includes)

In [ ]:
pdfmaker.remove_previous_actions()

In [ ]:
pdfmaker.generate_chordpro_song_pdfs()

In [ ]:
pdfmaker.copy_and_rename_pdf_songs()

### Merge and sort subfolders per section

- start custom sort for file name with hashtag #<sortstring><space><filename>
- implement in next notebook and latexgenerator class

In [ ]:
pdfmaker.save_configuration()

In [ ]:
# chordpro_output = song_sections[0]['songs'][0]['chordpro_output']

# chordpro_output

# print(chordpro_output["stderr"])

# pp.pprint(song_sections)

### backup